In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/data_assign_2

/content/drive/MyDrive/data_assign_2


In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import os


In [ ]:
# Define the path to your dataset
dataset_dir = 'hand_written_digits'



In [ ]:
def split_dataset(dataset_path, validation_split=0.2):
    classes = ['0_digits', '1_digits', '2_digits']

    train_data = []
    val_data = []

    for class_name in classes:
        class_path = os.path.join(dataset_path, class_name)
        image_list = os.listdir(class_path)
        train_images, val_images = train_test_split(image_list, test_size=validation_split, random_state=42)
        train_data.extend([(os.path.join(class_path, img), class_name) for img in train_images])
        val_data.extend([(os.path.join(class_path, img), class_name) for img in val_images])

    return train_data, val_data

train_data, val_data = split_dataset(dataset_dir)


In [ ]:
img_width, img_height = 100, 100
batch_size = 32

In [ ]:
# Data augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False,
    vertical_flip=False
)


In [ ]:
# Load the pre-trained VGG16 model without the top layer
base_model = VGG16(input_shape=(img_width, img_height, 3), include_top=False, weights='imagenet')

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(3, activation='softmax')(x)  # 3 classes for digits 0, 1, and 2


In [ ]:
# Create the fine-tuned model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])



In [ ]:
# Load and prepare the data
train_generator = datagen.flow_from_dataframe(dataframe=pd.DataFrame(train_data, columns=['filename', 'class']),
    x_col='filename',
    y_col='class',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

val_generator = datagen.flow_from_dataframe(dataframe=pd.DataFrame(val_data, columns=['filename', 'class']),
    x_col='filename',
    y_col='class',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 3725 validated image filenames belonging to 3 classes.
Found 933 validated image filenames belonging to 3 classes.


In [ ]:
# Train the fine-tuned model
epochs = 10
model.fit(train_generator, validation_data=val_generator, epochs=epochs)


Epoch 1/10
117/117 [==============================] - 855s 7s/step - loss: 0.8221 - accuracy: 0.8043 - val_loss: 0.5764 - val_accuracy: 0.9743
Epoch 2/10
117/117 [==============================] - 351s 3s/step - loss: 0.4338 - accuracy: 0.9764 - val_loss: 0.3193 - val_accuracy: 0.9807
Epoch 3/10
117/117 [==============================] - 352s 3s/step - loss: 0.2558 - accuracy: 0.9801 - val_loss: 0.2035 - val_accuracy: 0.9893
Epoch 4/10
117/117 [==============================] - 354s 3s/step - loss: 0.1728 - accuracy: 0.9809 - val_loss: 0.1504 - val_accuracy: 0.9818
Epoch 5/10
117/117 [==============================] - 353s 3s/step - loss: 0.1235 - accuracy: 0.9850 - val_loss: 0.1093 - val_accuracy: 0.9882
Epoch 6/10
117/117 [==============================] - 365s 3s/step - loss: 0.0996 - accuracy: 0.9863 - val_loss: 0.0902 - val_accuracy: 0.9839
Epoch 7/10
117/117 [==============================] - 355s 3s/step - loss: 0.0831 - accuracy: 0.9866 - val_loss: 0.0760 - val_accuracy: 0.9839

In [ ]:
# Evaluate the fine-tuned model
validation_accuracy = model.evaluate(val_generator)[1]
print(f'Validation accuracy (Fine-tuned VGG16 Model): {validation_accuracy}')

30/30 [==============================] - 72s 2s/step - loss: 0.0472 - accuracy: 0.9914
Validation accuracy (Fine-tuned VGG16 Model): 0.9914255142211914
